# Imports

In [1]:
!pip install transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessor, LogitsProcessorList, GPT2LMHeadModel, GPT2Tokenizer, PrefixConstrainedLogitsProcessor
from transformers.tokenization_utils import Trie
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors
import torch
# import openai

# Build Vocabulary
Creating a set of words to work as a vocabulary.

## Dolch word list

In [3]:
pre_primer = ["a", "and", "away", "big", "blue", "can", "come", "down", "find", "for", "funny", "go", "help", "here", "I", "in", "is", "it", "jump", "little", "look", "make", "me", "my", "not", "one", "play", "red", "run", "said", "see", "the", "three", "to", "two", "up", "we", "where", "yellow", "you"]
primer = ["all", "am", "are", "at", "ate", "be", "black", "brown", "but", "came", "did", "do", "eat", "four", "get", "good", "have", "he", "into", "like", "must", "new", "no", "now", "on", "our", "out", "please", "pretty", "ran", "ride", "saw", "say", "she", "so", "soon", "that", "there", "they", "this", "too", "under", "want", "was", "well", "went", "what", "white", "who", "will", "with", "yes"]
grade_1 = ["after", "again", "an", "any", "as", "ask", "by", "could", "every", "fly", "from", "give", "going", "had", "has", "her", "him", "his", "how", "just", "know", "let", "live", "may", "of", "old", "once", "open", "over", "put", "round", "some", "stop", "take", "thank", "them", "then", "think", "walk", "were", "when"]
grade_2 = ["always", "around", "because", "been", "before", "best", "both", "buy", "call", "cold", "does", "don't", "fast", "first", "five", "found", "gave", "goes", "green", "its", "made", "many", "off", "or", "pull", "read", "right", "sing", "sit", "sleep", "tell", "their", "these", "those", "upon", "us", "use", "very", "wash", "which", "why", "wish", "work", "would", "write", "your"]
grade_3 = ["about", "better", "bring", "carry", "clean", "cut", "done", "draw", "drink", "eight", "fall", "far", "full", "got", "grow", "hold", "hot", "hurt", "if", "keep", "kind", "laugh", "light", "long", "much", "myself", "never", "only", "own", "pick", "seven", "shall", "show", "six", "small", "start", "ten", "today", "together", "try", "warm"]
nouns = ["apple", "baby", "back", "ball", "bear", "bed", "bell", "bird", "birthday", "boat", "box", "boy", "bread", "brother", "cake", "car", "cat", "chair", "chicken", "children", "Christmas", "coat", "corn", "cow", "day", "dog", "doll", "door", "duck", "egg", "eye", "farm", "farmer", "father", "feet", "fire", "fish", "floor", "flower", "game", "garden", "girl", "goat", "grass", "ground", "hand", "head", "hill", "home", "horse", "house", "kitty", "leg", "letter", "man", "men", "milk", "money", "morning", "mother", "name", "nest", "night", "paper", "party", "picture", "pig", "rabbit", "rain", "ring", "robin", "Santa Claus", "school", "seed", "sheep", "shoe", "sister", "snow", "song", "squirrel", "stick", "street", "sun", "table", "thing", "time", "top", "toy", "tree", "watch", "water", "way", "wind", "window", "woman", "women", "wood"]

dolch_word_list = pre_primer + primer + grade_1 + grade_2 + grade_3 + nouns

In [4]:
# Vocabulary
vocabulary = dolch_word_list

In order to avoid splitting the words into smaller tokens, we need a word level tokenizer. This is because otherwise out of vocabulary words may be produced using tokens from the vocabulary.

In [5]:
# Initialize the tokenizer with an empty model
tokenizer = Tokenizer(models.WordLevel())

# Set up pre-tokenizer and decoder
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.decoder = decoders.WordPiece()

# Define the trainer with your vocabulary
trainer = trainers.WordLevelTrainer(vocab=dolch_word_list + ['<pad>', '<unk>', '.', ',', '!', '?', ';', ':'])

# Train the tokenizer
tokenizer.train_from_iterator(dolch_word_list, trainer)

Ignored unknown kwargs option vocab


In [6]:
# # Load GPT-2 tokenizer and model
# model_name = 'gpt2'
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# model = GPT2LMHeadModel.from_pretrained(model_name)

In [7]:
# # Your vocabulary
# vocabulary = dolch_word_list

# num_beams = 5

# # Tokenize vocabulary words and build the trie
# vocabulary_token_sequences = []
# for word in vocabulary:
#     tokens = tokenizer.encode(word, add_special_tokens=False)
#     vocabulary_token_sequences.append(tokens)

# trie = Trie()
# for token_seq in vocabulary_token_sequences:
#     trie.add(tuple(token_seq))

# # Initialize the constrained logits processor
# logits_processor = LogitsProcessorList()
# logits_processor.append(PrefixConstrainedLogitsProcessor(
#     prefix_allowed_tokens_fn=lambda batch_id, input_ids: [
#         token_seq[len(input_ids)] for token_seq in vocabulary_token_sequences
#         if len(token_seq) > len(input_ids) and token_seq[:len(input_ids)] == tuple(input_ids.tolist())
#     ] if any(
#         len(token_seq) > len(input_ids) and token_seq[:len(input_ids)] == tuple(input_ids.tolist())
#         for token_seq in vocabulary_token_sequences
#     ) else list(range(len(tokenizer)))  # Fallback to allow any token if no matches found
#     ,
#     num_beams=num_beams
# ))


# # Set the initial prompt
# prompt = "Once upon"
# input_ids = tokenizer.encode(prompt, return_tensors='pt')

# # Generate text
# output_ids = model.generate(
#     input_ids,
#     max_length=50,
#     logits_processor=logits_processor,
#     num_beams=num_beams,  # Adjust for desired trade-off between performance and speed
#     eos_token_id=tokenizer.eos_token_id,
#     pad_token_id=tokenizer.eos_token_id,
# )

# # Decode the generated tokens
# generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print(generated_text)

In [8]:
# # Collect all tokens required to represent the vocabulary words
# vocabulary_tokens = set()
# for word in vocabulary:
#     tokens = tokenizer.tokenize(word)
#     vocabulary_tokens.update(tokens)

vocabulary_tokens = vocabulary

# Convert tokens to token IDs
vocabulary_token_ids = set(tokenizer.convert_tokens_to_ids(list(vocabulary_tokens)))

# Include tokens for spaces and punctuation
space_token_ids = []
punctuation_tokens = ['.', ',', '!', '?', ';', ':']

for token, token_id in tokenizer.get_vocab().items():
    if token.startswith('Ġ'):
        space_token_ids.append(token_id)
    elif token in punctuation_tokens:
        space_token_ids.append(token_id)

vocabulary_token_ids.update(space_token_ids)

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'convert_tokens_to_ids'

In [ ]:
class VocabularyFilterLogitsProcessor(LogitsProcessor):
    def __init__(self, vocabulary_token_ids):
        self.vocabulary_token_ids = vocabulary_token_ids

    def __call__(self, input_ids, scores):
        # Create a mask over the vocabulary tokens
        vocab_mask = torch.full(scores.shape, False, dtype=torch.bool, device=scores.device)
        vocab_mask[..., list(self.vocabulary_token_ids)] = True
        # Set scores for tokens not in the vocabulary to negative infinity
        scores[..., ~vocab_mask] = -float('inf')
        return scores

In [ ]:
# Initialize the logits processor with your vocabulary
vocabulary_processor = VocabularyFilterLogitsProcessor(vocabulary_token_ids)
logits_processor = LogitsProcessorList([vocabulary_processor])

# Set the initial prompt
prompt = "Here is a sentence:"
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate text
output_ids = model.generate(
    input_ids,
    max_length=50,
    logits_processor=logits_processor,
    do_sample=True,
    temperature=0.3,
    top_k=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# Decode the generated tokens
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)